In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from community import community_louvain
from citations import citations_dict

In [2]:
data = pd.read_csv('data.csv')
del data['Unnamed: 0']

data = data.filter(['Authors', 'Author(s) ID', 'Title', 'Year', 'Cited by', 'DOI', 'Author Keywords', 'Index Keywords', 'Conference name'])

data['Auth_ID'] = None
data['Auth_Keywords'] = None
data['Ind_Keywords'] = None

for index in range(len(data)):

    try:
        data.at[index, 'Auth_ID'] = data.iloc[index, 1].split(';')[:-1]
    except:
        pass
    try:
        data.at[index, 'Auth_Keywords'] = data.iloc[index, 6].split('; ')
    except:
        pass
    try:
        data.at[index, 'Ind_Keywords'] = data.iloc[index, 7].split('; ')
    except:
        pass

data = data.filter(['Authors', 'Auth_ID', 'Title', 'Year', 'Cited by', 'DOI', 'Auth_Keywords', 'Ind_Keywords', 'Conference name'])
print(len(data), list(data))

538 ['Authors', 'Auth_ID', 'Title', 'Year', 'Cited by', 'DOI', 'Auth_Keywords', 'Ind_Keywords', 'Conference name']


In [3]:
'''
Filtering citation DOI's already existing in the database, obtaining cited authors
'''

data_copy = data.copy()
existing_DOIs = data['DOI'].tolist()
data['Citations'] = None
data['Cit_Updated'] = None

for index in range(len(data)):
    doi = data.iloc[index, 5]
    try:
        data.at[index, 'Citations'] = citations_dict[doi]
    except:
        pass
    
for index in range(len(data)):
    citations = data.iloc[index, 9]
    fin_citations = []
    try:
        for citation in citations:
            if citation in existing_DOIs:
                fin_citations.append(citation)
        if len(fin_citations) > 0:
            data.at[index, 'Cit_Updated'] = fin_citations
    except:
        pass

data = data.dropna(subset = ['Cit_Updated'])
data = data.reset_index()
data = data.filter(['Authors', 'Auth_ID', 'Title', 'Cited by', 'DOI', 'Auth_Keywords', 'Ind_Keywords', 'Cit_Updated'])

data['Cit_Authors'] = None

for index in range(len(data)):
    citations = data.iloc[index, 7]
    cit_authors = []
    cit_authors_final = []
    
    for citation in citations:
        row = data_copy.loc[data_copy['DOI'] == citation]

        for author in row.iloc[0, 1]:
            cit_authors.append(author)
            
    [cit_authors_final.append(x) for x in cit_authors if x not in cit_authors_final]
    data.at[index, 'Cit_Authors'] = cit_authors_final

data

,Authors,Auth_ID,Title,Cited by,DOI,Auth_Keywords,Ind_Keywords,Cit_Updated,Cit_Authors
0,"Sengupta S., Chowdhary A., Huang D., Kambhampa...","[57193487072, 57190399852, 9733190700, 7005346...",General Sum Markov Games for Strategic Detecti...,8.0,10.1007/978-3-030-32430-8_29,None,"[Computation theory, Decision theory, Digital ...","[10.1080/09540091.2020.1832960, 10.1109/COMST....","[57218648647, 57199819500, 57219443938, 559210..."
1,"Acquaviva J., Mahon M., Einfalt B., Laporta T.","[57199510001, 7102995401, 36680045300, 7004716...",Optimal cyber-defense strategies for advanced ...,2.0,10.1109/SRDS.2017.29,"[Advanced persistent threats, Cyber attack mod...","[Budget control, Computer crime, Continuous ti...",[10.1016/j.cose.2020.101933],"[57217176384, 57212669237, 56703794800]"
2,"Saha S., Vullikanti A., Halappanavar M.","[55445032300, 24077196600, 49661237100]",FlipNet: Modeling Covert and Persistent Attack...,1.0,10.1109/ICDCS.2017.298,"[Advanced persistent threats, FlipIt, Security...","[Computer games, Game theory, Advanced persist...",[10.4218/etrij.2019-0152],"[57210136318, 24537536000, 55264296800, 357718..."
3,"Laszka A., Horvath G., Felegyhazi M., Buttyán L.","[26638834900, 56284270400, 6507839126, 6701381...",FlipThem: Modeling targeted attacks with FlipI...,35.0,10.1007/978-3-319-12601-2_10,"[Advanced persistent threats, Attacker-defende...","[Artificial intelligence, Computer science, Co...","[10.1109/TIFS.2019.2955891, 10.1109/TIFS.2018....","[57164157500, 24767254400, 57218305777, 571893..."
4,"Johnson J.R., Hogan E.A.","[55494676400, 24171066500]",A graph analytic metric for mitigating advance...,24.0,10.1109/ISI.2013.6578801,"[cybersecurity, discrete mathematics, graph th...","[Cyber security, Discrete mathematics, Graph-a...","[10.1016/j.cose.2020.101734, 10.3390/s20030731...","[55602888600, 57197738184, 6507227812, 5719905..."
...,...,...,...,...,...,...,...,...,...
243,"Sayin M.O., Hosseini H., Poovendran R., Başar T.","[55832164500, 8130972800, 6603592481, 7102588845]",A game theoretical framework for inter-process...,4.0,10.1007/978-3-030-01554-1_28,"[Advanced persistent threats, Anomaly detectio...","[Decision theory, Intrusion detection, Network...",[10.1109/ICCPS48487.2020.00009],"[56641851400, 57205437689, 56405494100, 700355..."
244,"Pawlick J., Zhu Q.","[57038872900, 24767254400]",Strategic Trust in Cloud-Enabled Cyber-Physica...,30.0,10.1109/TIFS.2017.2725224,"[cyber-physical systems, cyber-security, Inter...","[Computation theory, Embedded systems, Game th...","[10.1109/TIFS.2018.2885251, 10.1109/TIFS.2018....","[57218305777, 57189377027, 55508709300, 353384..."
245,"Chen J., Zhu Q.","[57092363300, 24767254400]",Security as a Service for Cloud-Enabled Intern...,48.0,10.1109/TIFS.2017.2718489,"[advanced persistent threats, cloud computing,...","[Automation, Cloud computing, Computation theo...","[10.1109/TII.2019.2925035, 10.1109/TIFS.2019.2...","[56523759100, 55916672500, 15051213900, 226363..."
246,"Quintero-Bonilla S., del Rey A.M.","[57195303529, 56264675400]",Proposed models for advanced persistent threat...,1.0,10.1007/978-3-030-23946-6_16,"[Advanced persistent threats, Artificial intel...","[Artificial intelligence, Distributed computer...",[10.3390/app10113874],"[57195303529, 56264675400]"


In [4]:
'''
Creating the author graph
'''

auth_1 = nx.Graph()

for index in range(len(data)):
    authors = data.iloc[index, 1]
    for author in authors:
        if auth_1.has_node(author) == True:
            pass
        else:   
            auth_1.add_node(author)
            
for index in range(len(data)):
    authors = data.iloc[index, 1]
    cit_authors = data.iloc[index, 8]

    for author in authors:
        for cit_author in cit_authors:
            if author != cit_author:
                if auth_1.has_edge(author, cit_author) == True:
                    pass
                else:
                    auth_1.add_edge(author, cit_author)
                    
# Remove isolated nodes from graph

auth_1.remove_nodes_from(list(nx.isolates(auth_1)))
                    
part_1 = community_louvain.best_partition(auth_1)
num_part_1 = max(part_1.values()) + 1

print('number of partitions - {}'.format(num_part_1))

number of partitions - 17


In [7]:
author_df = pd.DataFrame()
author_df['Author ID'] = None
author_df['Partition'] = None
author_df['Keywords'] = None

index = 0
for key, value in part_1.items():
    author_df.at[index, 'Author ID'] = key
    author_df.at[index, 'Partition'] = value
    index += 1
    
for index in range(len(author_df)):
    author = author_df.iloc[index, 0]
    selection = [author]
    keywords = []
    keywords_final = []
    
    rows = data[data.Auth_ID.apply(lambda x: any(item for item in selection if item in x))]
    
    try:
        for j in range(len(rows)):
            for keyword in rows.iloc[j, 6]:
                keywords.append(keyword)
            
        [keywords_final.append(x) for x in keywords if x not in keywords_final]
        if len(keywords_final) > 0:
            author_df.at[index, 'Keywords'] = keywords_final
    except:
        pass
    
author_df = author_df.dropna(subset = ['Keywords'])

for index in range(num_part_1):
    df = author_df[author_df['Partition'] == index]
    author_dict = {}
    for i in range(len(df)):
        kwds = df.iloc[i, 2]
        try:
            for kwd in kwds:
                if kwd in author_dict.keys():
                    val = author_dict[kwd]
                    author_dict[kwd] = val + 1
                else:
                    author_dict[kwd] = 1
        except:
            pass
    max_key = max(author_dict, key = author_dict.get)
    print('Community - {}'.format(index))
    print('Community size - {}'.format(len(author_df[author_df['Partition'] == index])))
    print('Most Repeating Keyword - {}'.format(max_key))
    print('Most Repeating Keyword Count- {}\n'.format(author_dict[max_key]))
    
    for keyword in sorted(author_dict, key = author_dict.get, reverse = True)[:3]:
        print('Keyword - {}, Count - {}'.format(keyword, author_dict[keyword]))
    print('\n\n')

Community - 0
Community size - 53
Most Repeating Keyword - Computer crime
Most Repeating Keyword Count- 20

Keyword - Computer crime, Count - 20
Keyword - Security of data, Count - 19
Keyword - Network security, Count - 19



Community - 1
Community size - 7
Most Repeating Keyword - Artificial intelligence
Most Repeating Keyword Count- 7

Keyword - Artificial intelligence, Count - 7
Keyword - Computers, Count - 7
Keyword - Error detection, Count - 7



Community - 2
Community size - 2
Most Repeating Keyword - APT
Most Repeating Keyword Count- 2

Keyword - APT, Count - 2
Keyword - Hadoop, Count - 2
Keyword - HDFS, Count - 2



Community - 3
Community size - 98
Most Repeating Keyword - Network security
Most Repeating Keyword Count- 60

Keyword - Network security, Count - 60
Keyword - Computer crime, Count - 52
Keyword - Malware, Count - 44



Community - 4
Community size - 2
Most Repeating Keyword - Cloud computing
Most Repeating Keyword Count- 2

Keyword - Cloud computing, Count - 2
Key

In [14]:
# 0 - 
# Computer Crime - Network Security

# 1 - 
# Artificial Intelligence in error detection

# 2 - 
# Merge into 8 

# 3 - 
# Merge into 6

# 4 -
# Merge into 0

# 5 - 
# Merge into 11

# 6 - 
# Network Security - Computer Crime and Malware

# 7 - 
# Merge into 14

# 8 - 
# Big Data

# 9 - 
# Game Theory

# 10 - 
# Merge into 6

# 11 - 
# Network Security - Internet Protocol and Cyber Threats

# 12 - 
# Merge into 6

# 13 - 
# Malware - Financial Organisations

# 14 - 
# Computer Crime - Malware 

# 15 - 
# Merge into 14

# 16 - 
# Merge into 8


merge_dict = {2 : 8, 3 : 6, 4 : 0, 5 : 11, 7 : 14, 10 : 6, 12 : 6, 15 : 14, 16 : 8}

for key, value in merge_dict.items():
    author_df.loc[author_df[author_df['Partition'] == key].index.tolist(), 'Partition'] = value

order_values_dict = {6 : 2, 8 : 3, 9 : 4, 11 : 5, 13 : 6, 14 : 7}

for key, value in order_values_dict.items():
    author_df.loc[author_df[author_df['Partition'] == key].index.tolist(), 'Partition'] = value
        
author_df

,Author ID,Partition,Keywords
0,57193487072,2,"[Computation theory, Decision theory, Digital ..."
1,57190399852,2,"[Computation theory, Decision theory, Digital ..."
2,9733190700,2,"[Computation theory, Decision theory, Digital ..."
3,7005346665,2,"[Computation theory, Decision theory, Digital ..."
4,57199510001,4,"[Budget control, Computer crime, Continuous ti..."
...,...,...,...
667,57211376249,2,"[Computer crime, Crime, Cryptography, Intrusio..."
668,55832164500,2,"[Decision theory, Intrusion detection, Network..."
669,8130972800,2,"[Decision theory, Intrusion detection, Network..."
670,7102588845,2,"[Decision theory, Intrusion detection, Network..."


In [ ]:
auth_2 = nx.Graph()
part_2 = {}

for index in range(len(data)):
    authors = data.iloc[index, 1]
    for author in authors:
        if auth_2.has_node(author) == True:
            pass
        else:   
            auth_2.add_node(author)
            
for index in range(len(data)):
    authors = data.iloc[index, 1]
    cit_authors = data.iloc[index, 8]

    for author in authors:
        for cit_author in cit_authors:
            if author != cit_author:
                if auth_2.has_edge(author, cit_author) == True:
                    pass
                else:
                    auth_2.add_edge(author, cit_author)

author_list = author_df['Author ID'].tolist()

for node in list(auth_2):
    if node not in author_list:
        auth_2.remove_node(node)
                    
for index in range(len(author_df)): 
    part_2[author_df.iloc[index, 0]] = author_df.iloc[index, 1]
    
num_part_2 = max(part_2.values()) + 1
node_degree = dict(auth_2.degree)

plt.figure(figsize = (10, 10))

pos = nx.spring_layout(auth_2)
cmap = cm.get_cmap('viridis', num_part_2)
nx.draw_networkx_nodes(auth_2, pos, part_2.keys(), node_size = [x * 2 for x in node_degree.values()], cmap = cmap, node_color = list(part_2.values()))

plt.title('Author Graph')
plt.show()